In [49]:
import pandas as pd

# Load dataset
data = pd.read_csv("/content/traffic.csv")

In [50]:
data.head()

event        date       country      city  \
0  click  2021-08-21  Saudi Arabia    Jeddah   
1  click  2021-08-21  Saudi Arabia    Jeddah   
2  click  2021-08-21         India  Ludhiana   
3  click  2021-08-21        France   Unknown   
4  click  2021-08-21      Maldives      Malé   

                              artist          album          track  \
0                             Tesher    Jalebi Baby    Jalebi Baby   
1                             Tesher    Jalebi Baby    Jalebi Baby   
2                      Reyanna Maria      So Pretty      So Pretty   
3  Simone & Simaria, Sebastian Yatra  No Llores Más  No Llores Más   
4                             Tesher    Jalebi Baby    Jalebi Baby   

           isrc                                linkid  
0  QZNWQ2070741  2d896d31-97b6-4869-967b-1c5fb9cd4bb8  
1  QZNWQ2070741  2d896d31-97b6-4869-967b-1c5fb9cd4bb8  
2  USUM72100871  23199824-9cf5-4b98-942a-34965c3b0cc2  
3  BRUM72003904  35573248-4e49-47c7-af80-08a960fa74cd  
4  QZNWQ2070741  2d896d31-97b6-4869-967b-1c5fb9cd4bb8

## **Q1. Total and Daily Pageview Events**
a. How many total pageview events did the links in the dataset receive during the entire period?

In [51]:
total_pageviews = data[data['event'] == 'pageview'].shape[0]
print(f"Total Pageview Events: {total_pageviews}")

Total Pageview Events: 142015


b. What is the average number of pageview events per day?

In [52]:
daily_pageviews = data[data['event'] == 'pageview'].groupby('date').size()
print(f"Average Pageview Events Per Day: {daily_pageviews.mean()}")
print("Daily Pageview Events:")
print(daily_pageviews)

Average Pageview Events Per Day: 20287.85714285714
Daily Pageview Events:
date
2021-08-19    22366
2021-08-20    21382
2021-08-21    21349
2021-08-22    20430
2021-08-23    18646
2021-08-24    18693
2021-08-25    19149
dtype: int64


# **Q2. Analysis of Other Events**
a. What is the total count and distribution of other recorded events in the dataset?

In [53]:
event_distribution = data['event'].value_counts()
print("Event Distribution:")
print(event_distribution)

other_events = event_distribution.drop('pageview', errors='ignore')
print("Distribution of Other Events:")
print(other_events)

Event Distribution:
event
pageview    142015
click        55732
preview      28531
Name: count, dtype: int64
Distribution of Other Events:
event
click      55732
preview    28531
Name: count, dtype: int64


# Q3. Geographical Distribution
a. Which countries contributed to the pageviews?

In [54]:
pageviews_by_country = data[data['event'] == 'pageview']['country'].value_counts()
print("Countries Contributing to Pageviews:")
print(pageviews_by_country)

Countries Contributing to Pageviews:
country
Saudi Arabia         28873
India                27286
United States        20839
France                9674
Iraq                  4897
                     ...  
Wallis and Futuna        1
Solomon Islands          1
Guinea-Bissau            1
Lesotho                  1
Saint Martin             1
Name: count, Length: 211, dtype: int64


# **Q4. Click-Through Rate (CTR) Analysis**
a. What is the overall click-through rate (CTR) calculated as clicks/pageviews?

In [ ]:
clicks = data[data['event'] == ['click'].groupby('linkid').size()
pageviews = data[data['event'] == 'pageview'].groupby('linkid').size()

overall_ctr = clicks.sum() / pageviews.sum()
print(f"Overall Click-Through Rate (CTR): {overall_ctr}")

Overall Click-Through Rate (CTR): 0.3924374185825441


b. How does the CTR vary across different links?

In [56]:
ctr = (clicks / pageviews).fillna(0)
print("CTR Across Different Links:")
print(ctr)

CTR Across Different Links:
linkid
00073307-ae96-5089-a117-4783afb42f8e    0.0
00126b32-0c35-507b-981c-02c80d2aa8e7    1.0
0018cfff-50a1-5984-9715-01ef2d11a49a    0.0
0033934b-5d16-5a06-af58-d087bcdd3680    0.0
0034d6cf-3bd8-5ffe-aafc-b3959fc48608    0.0
                                       ... 
fff38ca0-8043-50cd-a5f1-f65ebb7105c5    1.0
fff4e5f0-4ee5-5fe7-aa30-e870edaf6ed7    0.0
fff84c0e-90a1-59d8-9997-adc909d50e16    1.0
fffc17a7-f935-5d3e-bd3e-d761fd80d479    0.5
fffd0045-29de-522b-b5d8-35786363bf07    0.5
Length: 3839, dtype: float64


# **Q5. Correlation Analysis**
a. Is there a correlation between clicks and previews on a link? If so, is it statistically significant, and how strong is the effect?

In [57]:
from scipy.stats import pearsonr

clicks = data[data['event'] == 'click'].groupby('linkid').size()
previews = data[data['event'] == 'preview'].groupby('linkid').size()

df = pd.DataFrame({'clicks': clicks, 'previews': previews}).fillna(0)
correlation, p_value = pearsonr(df['clicks'], df['previews'])
print(f"Pearson Correlation Coefficient: {correlation}")
print(f"P-value: {p_value}")


Pearson Correlation Coefficient: 0.9886810823208138
P-value: 0.0


b. Perform tests for both potential linear relationships and categorical (binary) relationships between these variables.

In [58]:
from scipy.stats import chi2_contingency

df['clicks_binary'] = (df['clicks'] > 0).astype(int)
df['previews_binary'] = (df['previews'] > 0).astype(int)

contingency_table = pd.crosstab(df['clicks_binary'], df['previews_binary'])
chi2, p, dof, expected = chi2_contingency(contingency_table)

print(f"Chi-Square Statistic: {chi2}")
print(f"P-value: {p}")
print(f"Degrees of Freedom: {dof}")
print("Expected Frequencies:")
print(expected)

Chi-Square Statistic: 109.07209677403016
P-value: 1.5648907281614155e-25
Degrees of Freedom: 1
Expected Frequencies:
[[  24.28740157    6.71259843]
 [1766.71259843  488.28740157]]


The Pearson coefficient (0.9887, p=0.0) shows a strong linear relationship between clicks and previews. Chi-Square (109.07, p=1.56e-25) indicates a significant association between binary occurrences.